<a href="https://colab.research.google.com/github/magotronico/DataAnalysis_and_AI/blob/main/IrisCalssifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Este codigo hace la clasificacion de 3 species de iris...

Equipo 2

In [2]:
import csv
import math
import random

def get_species(df):
    species = []
    for row in df:
        if row[4] not in species:
            species.append(row[4])
    return species

def calculate_mean(values):
    return sum(values) / len(values)

def calculate_std_dev(values, mean):
    variance = sum((x - mean) ** 2 for x in values) / (len(values) - 1)
    return math.sqrt(variance)

def calculate_mean_std(values):
    mean = calculate_mean(values)
    std_dev = calculate_std_dev(values, mean)
    return mean, std_dev

def range_per_species(df, species_list):
    ranges = {}
    for species in species_list:
        species_ranges = {}
        for i in range(4):  # 4 parameters (sepal length, sepal width, petal length, petal width)
            values = [float(row[i]) for row in df if row[4] == species]
            mean, std_dev = calculate_mean_std(values)
            species_ranges[f'param_{i+1}'] = (mean - std_dev, mean + std_dev)
        ranges[species] = species_ranges
    return ranges

def evaluate_classification(df, species_ranges):
    results = {species: {f'param_{i+1}': {"correct": 0, "incorrect": 0} for i in range(4)} for species in species_ranges}
    for row in df:
        actual_species = row[4]
        for i in range(4):  # 4 parameters
            param_range = species_ranges[actual_species][f'param_{i+1}']
            if param_range[0] <= float(row[i]) <= param_range[1]:
                results[actual_species][f'param_{i+1}']["correct"] += 1
            else:
                results[actual_species][f'param_{i+1}']["incorrect"] += 1
    return results

def print_results(results):
    for species, params in results.items():
        print(f"Species: {species}")
        for param, counts in params.items():
            print(f"  {param}:")
            print(f"    Correctly classified: {counts['correct']}")
            print(f"    Incorrectly classified: {counts['incorrect']}")
        total_correct = sum(counts['correct'] for counts in params.values())
        total_incorrect = sum(counts['incorrect'] for counts in params.values())
        print(f"  Total Correctly classified: {total_correct}")
        print(f"  Total Incorrectly classified: {total_incorrect}")
        print(f"  Total: {total_correct + total_incorrect}\n")

if __name__ == '__main__':
    df = []

    with open('Iris.csv', newline='') as csvfile:
        spamreader = csv.reader(csvfile, delimiter=',', quotechar='|')
        next(spamreader)  # Skip the header row
        for row in spamreader:
            df.append(row[1:])

    species_list = get_species(df)
    species_ranges = range_per_species(df, species_list)
    results = evaluate_classification(df, species_ranges)
    print_results(results)


Species: Iris-setosa
  param_1:
    Correctly classified: 31
    Incorrectly classified: 19
  param_2:
    Correctly classified: 32
    Incorrectly classified: 18
  param_3:
    Correctly classified: 40
    Incorrectly classified: 10
  param_4:
    Correctly classified: 35
    Incorrectly classified: 15
  Total Correctly classified: 138
  Total Incorrectly classified: 62
  Total: 200

Species: Iris-versicolor
  param_1:
    Correctly classified: 35
    Incorrectly classified: 15
  param_2:
    Correctly classified: 33
    Incorrectly classified: 17
  param_3:
    Correctly classified: 37
    Incorrectly classified: 13
  param_4:
    Correctly classified: 35
    Incorrectly classified: 15
  Total Correctly classified: 140
  Total Incorrectly classified: 60
  Total: 200

Species: Iris-virginica
  param_1:
    Correctly classified: 35
    Incorrectly classified: 15
  param_2:
    Correctly classified: 35
    Incorrectly classified: 15
  param_3:
    Correctly classified: 35
    Incorrectl

Aqui se observa que los parametros con mayor cantidad de clasificados correctos se reptiten entre las species y se ordenan de mayor a menor como:

Les pusimos pesos arbitrarios de:


definimos un umbral de 65% arbitrario que en el futuro mejorara al aprender mejores tecnicas

In [4]:
def test_random_records(df, species_ranges, num_records=100, umbral=0.65):
  random_records = random.sample(df, min(num_records, len(df)))

  # Define weights
  weights = {
      'param_1': 0.17,
      'param_2': 0.13,
      'param_3': 0.38,
      'param_4': 0.32
  }

  correct_count = 0
  incorrect_count = 0

  for record in random_records:
      species = record[4]
      if species not in species_ranges:
          continue

      weighted_score = 0
      total_weight = 0

      for i in range(4):  # 4 parameters
          param_key = f'param_{i+1}'
          param_value = float(record[i])
          param_range = species_ranges[species][param_key]

          if param_range[0] <= param_value <= param_range[1]:
              weighted_score += weights[param_key]

          total_weight += weights[param_key]

      score_percentage = weighted_score / total_weight

      if score_percentage >= umbral:
          correct_count += 1
      else:
          incorrect_count += 1

  total_records = correct_count + incorrect_count
  confiability = (correct_count / total_records) * 100 if total_records > 0 else 0

  print(f"Number of Correctly Classified Records: {correct_count}")
  print(f"Number of Incorrectly Classified Records: {incorrect_count}")
  print(f"Confiability: {confiability:.2f}%")

if __name__ == '__main__':
  df = []

  with open('Iris.csv', newline='') as csvfile:
      spamreader = csv.reader(csvfile, delimiter=',', quotechar='|')
      next(spamreader)  # Skip the header row
      for row in spamreader:
          df.append(row[1:])

  species_list = get_species(df)
  species_ranges = range_per_species(df, species_list)
  results = evaluate_classification(df, species_ranges)

  # Test 100 random records
  test_random_records(df, species_ranges)

Number of Correctly Classified Records: 67
Number of Incorrectly Classified Records: 33
Confiability: 67.00%
